In [17]:
import sys
import os
import django
from asgiref.sync import sync_to_async
import asyncio
from datetime import datetime, timezone
from datetime import timedelta
import numpy as np

from collections import defaultdict

# Absolute path to your Django project root (adjust this!)
project_path = "/home/pdapp/pd_api_server/"
sys.path.append(project_path)

# Set Django settings
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mysite.settings")

# Setup Django
django.setup()

from django.contrib.auth.models import User
from api.models import Patient


# Make ORM calls async-safe
@sync_to_async
def get_patient(pid):
    patient = Patient.objects.get(patientId=pid)
    return patient

@sync_to_async
def get_pid():
    return list(Patient.objects.values_list('patientId', flat=True))

@sync_to_async
def get_patient_related_data(patient):
    user = User.objects.get(username=patient.user_name)
    
    data = {
        "date_join":user.date_joined,
        "patient": {
            "id": patient.patientId,
            "name": patient.name,
            "user_name": patient.user_name,
            "email": patient.email,
            "phone_no": patient.phone_no,
            "id_no": patient.id_no,
            "gender": patient.gender,
            "age": patient.age,
            "birthday": patient.birthday,
        },
        "records": [],
        "uploaded_files": [],
        "results": [],
        "questionnaires": [],
    }

    for r in patient.patientrecord_set.all():
        data["records"].append({
            "time": r.time,
            "taking_pd_medicine": r.taking_pd_medicine,
            "taking_pd_med3hr": r.taking_pd_med3hr,
        })

    for f in patient.fileuploaded_set.all():
        data["uploaded_files"].append({
            "upload_time": f.upload_time,
            "file_type": f.file_type,
            "file_path": f.file_path,
        })

    for res in patient.results_set.all():
        data["results"].append({
            "upload_time": res.upload_time,
            "gait_result": res.gait_result,
            "voice_result": res.voice_result,
            "hand_result": res.hand_result,
            "multimodal_results": res.multimodal_results,
        })

    for q in patient.patientquestionairerecord_set.all():
        data["questionnaires"].append({
            "time": q.time,
            "riskMarker": q.riskMarker,
            "PLR": q.PLR,
            "TELR": q.TELR,
            "PostProb": q.PostProb,
            "PPPD": q.PPPD,
            "response": q.response,
        })

    return data


def get_1st_upload(data):
    create_date = data["date_join"] 
    upload_files_ls = data['uploaded_files']
    record_1st_datetime = []

    record_1st_datetime.append(create_date)

    for file_uploaded in upload_files_ls:
        if file_uploaded['file_type'] == "left_hand":
            record_1st_datetime.append(file_uploaded["upload_time"])
            break

    for file_uploaded in upload_files_ls:
        if file_uploaded['file_type'] == "right_hand":
            record_1st_datetime.append(file_uploaded["upload_time"])
            break
    
    for file_uploaded in upload_files_ls:
        if file_uploaded['file_type'] == "sound":
            record_1st_datetime.append(file_uploaded["upload_time"])
            break

    for file_uploaded in upload_files_ls:
        if file_uploaded['file_type'] == "gait":
            record_1st_datetime.append(file_uploaded["upload_time"])
            break

    for file_uploaded in upload_files_ls:
        if file_uploaded['file_type'] == "three":
            record_1st_datetime.append(file_uploaded["upload_time"])
            break

    for file_uploaded in upload_files_ls:
        if file_uploaded['file_type'] == "spiral_paint" and "right" in file_uploaded['file_path']:
            record_1st_datetime.append(file_uploaded["upload_time"])
            break
            
    for file_uploaded in upload_files_ls:
        if file_uploaded['file_type'] == "spiral_paint" and "left" in file_uploaded['file_path']:
            record_1st_datetime.append(file_uploaded["upload_time"])
            break
            

    record_1st_datetime.append(datetime.strptime(data["questionnaires"][0]["time"], "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc))
    
    return record_1st_datetime
                               

def get_duration(record_1st_datetime, out_data):

    record_duration = []

    for i in range(len(record_1st_datetime)):
        if i+1 < len(record_1st_datetime):
            record_duration.append(record_1st_datetime[i+1] -  record_1st_datetime[i])

    out_data["left_hand_record_duration"] =  record_duration[0]
    out_data["right_hand_record_duration"] =  record_duration[1]
    out_data["sound_hand_record_duration"] =  record_duration[2]
    out_data["gait_record_duration"] =  record_duration[3]
    out_data["three_paint_record_duration"] =  record_duration[4]
    out_data["right_spiral_paint_record_duration"] =  record_duration[5]
    out_data["left_spiral_paint_record_duration"] =  record_duration[6]
    out_data["questionnaires"] = record_duration[7]

    return out_data

def get_whole_duration(data):
    end_point = datetime.strptime(data['results'][0]["upload_time"], "%Y-%m-%d_%H:%M:%S")
    created_point = data["date_join"] 

    # Make it timezone-aware (UTC)
    aware_naive_dt = end_point.replace(tzinfo=timezone.utc)

    return aware_naive_dt - created_point 


In [2]:
remove_list = [2,10,11,12,15,29,35,45,63]

all_pid = await get_pid()

all_pid_duration = []
all_out_data = []

for remove_pid in remove_list:
    all_pid.remove(remove_pid)

for pid in all_pid:
    out_data = {}
    patient = await get_patient(str(pid))
    out_data["user_name"] = patient.user_name
    data = await get_patient_related_data(patient)
    
    try:
        whole_duration = get_whole_duration(data)
    except Exception as e:
        whole_duration = None
        print(e)
        
    all_pid_duration.append(whole_duration)
    
    try:
        upload_times = get_1st_upload(data)
        out_data = get_duration(upload_times, out_data)
    
    except Exception as e:
        out_data = {}
        print(patient.patientId)
        print(e)
        
    all_out_data.append(out_data)

list index out of range
40
list index out of range
list index out of range
41
list index out of range
list index out of range
42
list index out of range
list index out of range
47
list index out of range
list index out of range
list index out of range
53
list index out of range
55
list index out of range
list index out of range
56
list index out of range
59
list index out of range
60
list index out of range
61
list index out of range
62
list index out of range
64
list index out of range
65
list index out of range
66
list index out of range
67
list index out of range
68
list index out of range
69
list index out of range
70
list index out of range
71
list index out of range
72
list index out of range
73
list index out of range
74
list index out of range
75
list index out of range
list index out of range
92
list index out of range
list index out of range
112
list index out of range
list index out of range
115
list index out of range
list index out of range
list index out of range
127
list

DoesNotExist: User matching query does not exist.

In [3]:
patient = await get_patient("32")
print(patient.user_name)
data = await get_patient_related_data(patient)
upload_times = get_1st_upload(data)

NTUH002_20250310_3


In [4]:
upload_times

[datetime.datetime(2025, 3, 10, 3, 10, 9, 775420, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 3, 14, 27, 66802, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 3, 15, 22, 540029, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 3, 17, 0, 504508, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 3, 22, 45, 177349, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 3, 24, 8, 344112, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 3, 24, 31, 465322, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 3, 24, 46, 100033, tzinfo=datetime.timezone.utc),
 datetime.datetime(2025, 3, 10, 11, 28, 51, tzinfo=datetime.timezone.utc)]

In [5]:
all_pid_duration

# Subtract 8 hours (28800 seconds)
eight_hours = timedelta(hours=8)

# Filter + Adjust
filtered = [
    t - eight_hours
    for t in all_pid_duration
    if t is not None and t.days == 0
]
deltas_in_minutes = [round(t.total_seconds() / 60, 2) for t in filtered]

# Result

np.mean(np.array(deltas_in_minutes)), np.std(np.array(deltas_in_minutes))

np.max(np.array(deltas_in_minutes))

43.18

In [29]:
import os
from collections import defaultdict
from django.contrib.auth.models import User
from django.core.exceptions import ObjectDoesNotExist
from django.core.files.storage import FileSystemStorage
from asgiref.sync import sync_to_async

# --- Storage configuration ---
base_path = '/mnt/pd_app'
sound_storage = FileSystemStorage(location=f'{base_path}/sound')
walk_storage = FileSystemStorage(location=f'{base_path}/walk')
gesture_storage = FileSystemStorage(location=f'{base_path}/gesture')
paint_spiral_right_storage = FileSystemStorage(location=f'{base_path}/paint/spiral/right')
paint_spiral_left_storage = FileSystemStorage(location=f'{base_path}/paint/spiral/left')
paint_three_right_storage = FileSystemStorage(location=f'{base_path}/paint/three/right')
paint_three_left_storage = FileSystemStorage(location=f'{base_path}/paint/three/left')
result_storage = FileSystemStorage(location=f'{base_path}/results')

# --- Utilities ---
def get_file_size_safe(path):
    try:
        return os.path.getsize(path)
    except FileNotFoundError:
        print(f"[Missing] File not found: {path}")
        return 0

def get_full_file_path(f):
    if f.file_type == "sound":
        return os.path.join(sound_storage.location, f.file_path)
    elif f.file_type == "gait":
        return os.path.join(walk_storage.location, f.file_path)
    elif f.file_type == "right_hand":
        return os.path.join(gesture_storage.location, f.file_path)
    elif f.file_type == "left_hand":
        return os.path.join(gesture_storage.location, f.file_path)
    elif f.file_type == "three":
        return os.path.join(paint_spiral_right_storage.location, f.file_path)
    elif f.file_type == "spiral_paint":
        return os.path.join(paint_spiral_right_storage.location, f.file_path)
    elif f.file_type == "result":
        return os.path.join(result_storage.location, f.file_path)
    else:
        return None

# --- Output containers ---
pid_by_month = defaultdict(list)
file_size_by_month = defaultdict(int)  # in bytes

# --- Main loop ---
for pid in all_pid:
    patient = await get_patient(str(pid))
    try:
        user = await sync_to_async(User.objects.get)(username=patient.user_name)
        join_month = user.date_joined.strftime('%Y-%m')
        pid_by_month[join_month].append(user.id)

        # Fetch uploaded files
        uploaded_files = await sync_to_async(list)(patient.fileuploaded_set.all())
        for f in uploaded_files:
            file_path = get_full_file_path(f)
            if file_path is None:
                print(f"[Warning] Unknown file type: {f.file_type}")
                continue
            file_size = await sync_to_async(get_file_size_safe)(file_path)
            file_size_by_month[join_month] += file_size

    except User.DoesNotExist:
        print(f"[Warning] User not found for patient: {patient.user_name}")

# --- Optional: human-readable display ---
def human_readable(size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

print("\n=== File Size Summary by Month ===")
for month in sorted(file_size_by_month):
    print(f"{month}: {human_readable(file_size_by_month[month])}")



[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 09:36:09_three_right_NTUH002_20250303_1__17_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 09:37:04_spiral_left_NTUH002_20250303_1__17_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:31:06_three_right_NTUH002_20250303_2__18_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:31:49_spiral_left_NTUH002_20250303_2__18_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:49:11_three_right_NTUH002_20250303_3__19_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:50:04_spiral_left_NTUH002_20250303_3__19_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 12:16:57_three_right_NTUH002_20250303_4__20_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 12:17:48_spiral_left_NTUH002_20250303_4__20_image.png
[Missing] File not found

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-17 15:00:45_spiral_left_002-20250317-10__55_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 16:21:30_three_right_008_20250319_1__57_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 16:23:21_spiral_left_008_20250319_1__57_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 17:01:58_three_right_008_20250319_2__58_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 17:03:28_spiral_left_008_20250319_2__58_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-26 15:04:35_three_right_20250326_143045__59_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-26 15:05:40_spiral_left_20250326_143045__59_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-26 15:06:00_spiral_left_20250326_143045__59_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/r

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-02 16:27:07_three_right_20250402_161217__80_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-02 16:29:40_spiral_left_20250402_161217__80_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 09:47:03_three_right_20250407_092454__81_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 09:48:47_spiral_left_20250407_092454__81_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 11:10:30_three_right_20250407_103556__82_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 11:11:42_spiral_left_20250407_103556__82_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 11:58:46_three_right_20250407_114410__83_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 12:00:26_spiral_left_20250407_114410__83_image.png
[Missing] File not found: /mnt/pd_app/paint/spir

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 14:59:58_three_right_20250414_144804__110_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 15:01:13_spiral_left_20250414_144804__110_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 16:01:24_three_right_20250414_155148__111_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 16:02:20_spiral_left_20250414_155148__111_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 14:19:30_three_right_20250415_135222__112_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 14:20:09_spiral_left_20250415_135222__112_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 15:39:27_three_right_20250415_151810__113_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 15:39:50_spiral_left_20250415_151810__113_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-23 15:33:03_three_right_20250423_151826__140_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-23 15:34:44_spiral_left_20250423_151826__140_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:02:08_three_right_20250424_090936__141_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:02:55_spiral_left_20250424_090936__141_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:24:01_three_right_20250424_110902__142_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:25:06_spiral_left_20250424_110902__142_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:54:06_three_right_20250424_113908__143_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:55:02_spiral_left_20250424_113908__143_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-12 11:52:40_three_right_20250512_112822__177_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-12 11:54:35_spiral_left_20250512_112822__177_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-12 12:11:07_three_right_20250512_115804__178_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-12 12:12:07_spiral_left_20250512_115804__178_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-12 12:35:44_three_right_20250512_121813__179_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-12 12:37:17_spiral_left_20250512_121813__179_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-13 11:09:59_three_right_20250512_145234__180_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-13 11:10:54_spiral_left_20250512_145234__180_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:12:48_three_right_20250526_094911__213_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:13:41_spiral_left_20250526_094911__213_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:35:13_three_right_20250526_101937__214_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:36:07_spiral_left_20250526_101937__214_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:10:37_three_right_20250526_105133__215_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:11:58_spiral_left_20250526_105133__215_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:27:45_three_right_20250526_111630__216_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:28:29_spiral_left_20250526_111630__216_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 11:24:14_three_right_20250605_111051__248_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 11:24:51_spiral_left_20250605_111051__248_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:01:11_three_right_20250605_113336__249_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:02:51_spiral_left_20250605_113336__249_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:52:44_three_right_20250605_123237__250_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:53:52_spiral_left_20250605_123237__250_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 13:16:38_three_right_20250605_130255__251_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 13:17:27_spiral_left_20250605_130255__251_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:04:35_three_right_20250623_104719__289_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:05:46_spiral_left_20250623_104719__289_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:27:48_three_right_20250623_111449__290_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:28:40_spiral_left_20250623_111449__290_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:48:57_three_right_20250623_113618__291_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:50:02_spiral_left_20250623_113618__291_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 12:15:57_three_right_20250623_115905__292_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 12:17:37_spiral_left_20250623_115905__292_image.png
[Missing] File not found: /mnt/pd_app/pa

[Warning] User not found for patient: 20250710_100939
[Warning] User not found for patient: 20250710_114823
[Warning] User not found for patient: 20250710_114913
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 13:33:41_three_right_20250710_131739__337_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 13:34:53_spiral_left_20250710_131739__337_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 14:51:11_three_right_20250710_142241__338_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 14:52:25_spiral_left_20250710_142241__338_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-11 14:09:20_three_right_20250711_135647__339_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-11 14:10:47_spiral_left_20250711_135647__339_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-11 15:16:37_three_right_20250711_150053__340_image.pn

[<FileUploaded: 20250721_133024_gait>,
 <FileUploaded: 20250721_133024_left_hand>,
 <FileUploaded: 20250721_133024_left_hand>,
 <FileUploaded: 20250721_133024_left_hand>,
 <FileUploaded: 20250721_133024_right_hand>,
 <FileUploaded: 20250721_133024_sound>,
 <FileUploaded: 20250721_133024_sound>,
 <FileUploaded: 20250721_133024_three>,
 <FileUploaded: 20250721_133024_spiral_paint>,
 <FileUploaded: 20250721_133024_spiral_paint>]

In [33]:
file_size_per_pid = defaultdict(int)     # total bytes per pid
file_count_per_pid = defaultdict(int)    # file count per pid

for pid in all_pid:
    patient = await get_patient(str(pid))
    try:
        user = await sync_to_async(User.objects.get)(username=patient.user_name)
        join_month = user.date_joined.strftime('%Y-%m')
        pid_by_month[join_month].append(user.id)

        # Fetch uploaded files
        uploaded_files = await sync_to_async(list)(patient.fileuploaded_set.all())
        for f in uploaded_files:
            file_path = get_full_file_path(f)
            if file_path is None:
                print(f"[Warning] Unknown file type: {f.file_type}")
                continue
            file_size = await sync_to_async(get_file_size_safe)(file_path)
            file_size_by_month[join_month] += file_size

            # Track per-pid
            file_size_per_pid[pid] += file_size
            file_count_per_pid[pid] += 1

    except User.DoesNotExist:
        print(f"[Warning] User not found for patient: {patient.user_name}")
def human_readable(size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

print("\n=== Average File Size Per PID ===")
for pid in sorted(file_size_per_pid):
    count = file_count_per_pid[pid]
    if count == 0:
        avg = 0
    else:
        avg = file_size_per_pid[pid] / count
    print(f"PID {pid}: {human_readable(avg)} ({count} files)")



[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 09:36:09_three_right_NTUH002_20250303_1__17_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 09:37:04_spiral_left_NTUH002_20250303_1__17_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:31:06_three_right_NTUH002_20250303_2__18_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:31:49_spiral_left_NTUH002_20250303_2__18_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:49:11_three_right_NTUH002_20250303_3__19_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 11:50:04_spiral_left_NTUH002_20250303_3__19_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 12:16:57_three_right_NTUH002_20250303_4__20_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-03 12:17:48_spiral_left_NTUH002_20250303_4__20_image.png
[Missing] File not found

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-17 15:00:45_spiral_left_002-20250317-10__55_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 16:21:30_three_right_008_20250319_1__57_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 16:23:21_spiral_left_008_20250319_1__57_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 17:01:58_three_right_008_20250319_2__58_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-19 17:03:28_spiral_left_008_20250319_2__58_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-26 15:04:35_three_right_20250326_143045__59_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-26 15:05:40_spiral_left_20250326_143045__59_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-03-26 15:06:00_spiral_left_20250326_143045__59_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/r

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-02 16:27:07_three_right_20250402_161217__80_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-02 16:29:40_spiral_left_20250402_161217__80_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 09:47:03_three_right_20250407_092454__81_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 09:48:47_spiral_left_20250407_092454__81_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 11:10:30_three_right_20250407_103556__82_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 11:11:42_spiral_left_20250407_103556__82_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 11:58:46_three_right_20250407_114410__83_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-07 12:00:26_spiral_left_20250407_114410__83_image.png
[Missing] File not found: /mnt/pd_app/paint/spir

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 14:59:58_three_right_20250414_144804__110_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 15:01:13_spiral_left_20250414_144804__110_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 16:01:24_three_right_20250414_155148__111_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-14 16:02:20_spiral_left_20250414_155148__111_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 14:19:30_three_right_20250415_135222__112_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 14:20:09_spiral_left_20250415_135222__112_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 15:39:27_three_right_20250415_151810__113_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-15 15:39:50_spiral_left_20250415_151810__113_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-23 15:33:03_three_right_20250423_151826__140_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-23 15:34:44_spiral_left_20250423_151826__140_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:02:08_three_right_20250424_090936__141_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:02:55_spiral_left_20250424_090936__141_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:24:01_three_right_20250424_110902__142_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:25:06_spiral_left_20250424_110902__142_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:54:06_three_right_20250424_113908__143_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-04-24 11:55:02_spiral_left_20250424_113908__143_image.png
[Missing] File not found: /mnt/pd_app/pa

...

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:12:48_three_right_20250526_094911__213_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:13:41_spiral_left_20250526_094911__213_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:35:13_three_right_20250526_101937__214_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 10:36:07_spiral_left_20250526_101937__214_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:10:37_three_right_20250526_105133__215_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:11:58_spiral_left_20250526_105133__215_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:27:45_three_right_20250526_111630__216_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-05-26 11:28:29_spiral_left_20250526_111630__216_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 11:24:14_three_right_20250605_111051__248_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 11:24:51_spiral_left_20250605_111051__248_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:01:11_three_right_20250605_113336__249_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:02:51_spiral_left_20250605_113336__249_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:52:44_three_right_20250605_123237__250_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 12:53:52_spiral_left_20250605_123237__250_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 13:16:38_three_right_20250605_130255__251_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-05 13:17:27_spiral_left_20250605_130255__251_image.png
[Missing] File not found: /mnt/pd_app/pa

[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:04:35_three_right_20250623_104719__289_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:05:46_spiral_left_20250623_104719__289_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:27:48_three_right_20250623_111449__290_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:28:40_spiral_left_20250623_111449__290_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:48:57_three_right_20250623_113618__291_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 11:50:02_spiral_left_20250623_113618__291_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 12:15:57_three_right_20250623_115905__292_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-06-23 12:17:37_spiral_left_20250623_115905__292_image.png
[Missing] File not found: /mnt/pd_app/pa

[Warning] User not found for patient: 20250710_100939
[Warning] User not found for patient: 20250710_114823
[Warning] User not found for patient: 20250710_114913
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 13:33:41_three_right_20250710_131739__337_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 13:34:53_spiral_left_20250710_131739__337_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 14:51:11_three_right_20250710_142241__338_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-10 14:52:25_spiral_left_20250710_142241__338_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-11 14:09:20_three_right_20250711_135647__339_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-11 14:10:47_spiral_left_20250711_135647__339_image.png
[Missing] File not found: /mnt/pd_app/paint/spiral/right/2025-07-11 15:16:37_three_right_20250711_150053__340_image.pn

PID 174: 5.66 MB (14 files)
PID 175: 6.08 MB (11 files)
PID 176: 7.09 MB (10 files)
PID 177: 4.60 MB (10 files)
PID 178: 4.70 MB (10 files)
PID 179: 5.87 MB (10 files)
PID 180: 6.13 MB (17 files)
PID 181: 4.14 MB (12 files)
PID 182: 4.83 MB (12 files)
PID 183: 4.84 MB (12 files)
PID 184: 18.55 MB (7 files)
PID 185: 6.99 MB (14 files)
PID 186: 5.80 MB (14 files)
PID 187: 5.17 MB (14 files)
PID 188: 7.95 MB (14 files)
PID 189: 6.87 MB (14 files)
PID 190: 6.44 MB (14 files)
PID 191: 5.29 MB (14 files)
PID 192: 8.07 MB (10 files)
PID 193: 5.34 MB (10 files)
PID 194: 5.67 MB (10 files)
PID 195: 5.63 MB (10 files)
PID 196: 8.36 MB (10 files)
PID 197: 3.36 MB (10 files)
PID 198: 5.82 MB (10 files)
PID 199: 5.44 MB (10 files)
PID 200: 5.89 MB (14 files)
PID 201: 5.33 MB (14 files)
PID 202: 6.18 MB (14 files)
PID 203: 4.97 MB (12 files)
PID 205: 5.35 MB (12 files)
PID 206: 6.12 MB (15 files)
PID 207: 5.72 MB (15 files)
PID 208: 7.00 MB (14 files)
PID 209: 6.59 MB (14 files)
PID 210: 5.76 MB (14

In [22]:
pid_03 = pid_by_month["2025-03"]

for pid in pid_03:
    patient = await get_patient(str(pid))

[34,
 35,
 36,
 37,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 73,
 74,
 75,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93]

In [6]:
# emtpy_patient 40, 41, 42, 53, 56, 89

In [7]:
# 47 didnt change hand 

In [8]:
data['results'][0]

{'upload_time': '2025-03-10_11:22:48',
 'gait_result': '56.082125520648304',
 'voice_result': '72.7260891646205',
 'hand_result': '43.73258462356281',
 'multimodal_results': '56.69029704313817'}

In [12]:
right_hand = []
left_hand = []
sound = []
gait = []
three = []
spiral_right = []
spiral_left = []
ques = []

for item in all_out_data:
    if item:
        left_hand.append(item["left_hand_record_duration"])
        right_hand.append(item["right_hand_record_duration"])
        sound.append(item["sound_hand_record_duration"])
        gait.append(item["gait_record_duration"])
        three.append(item["three_paint_record_duration"])
        spiral_left.append(item["left_spiral_paint_record_duration"])
        spiral_right.append(item["right_spiral_paint_record_duration"])
        ques.append(item["questionnaires"])

# Subtract 8 hours (28800 seconds)
eight_hours = timedelta(hours=8)

# Filter + Adjust
ques = [
    t - eight_hours
    for t in ques
    if t is not None and t.days == 0
]

In [15]:
minutes_timedelta = lambda x: [round(t.total_seconds() / 60, 2) for t in x]
get_second = lambda x: [int(t.total_seconds()) for t in x]

def mean_std_min_max(duration_ls, display="Right Hand"):
    arr = np.array(get_second(duration_ls))
    mean = np.mean(arr)
    std = np.std(arr)
    _min = np.min(arr)
    _max = np.max(arr) 

    print(f"{display}: mean = {mean:.2f}s, std = {std:.2f}s, min = {_min}s ~ max = {_max}s")

mean_std_min_max(left_hand, display="Left Hand")
mean_std_min_max(right_hand, display="Right Hand")
mean_std_min_max(sound, display="Sound")
mean_std_min_max(gait, display="Gait")
mean_std_min_max(three, display="Paint three")
mean_std_min_max(spiral_left, display="Spiral left")
mean_std_min_max(spiral_right, display="Spiral right")
mean_std_min_max(ques, display="questionnaires")

Left Hand: mean = 315.61s, std = 327.36s, min = 68s ~ max = 1301s
Right Hand: mean = 74.13s, std = 171.40s, min = 23s ~ max = 1194s
Sound: mean = 91.85s, std = 33.97s, min = 56s ~ max = 221s
Gait: mean = 416.22s, std = 132.92s, min = -186s ~ max = 700s
Paint three: mean = 95.98s, std = 64.81s, min = 58s ~ max = 509s
Spiral left: mean = 36.74s, std = 35.64s, min = 14s ~ max = 234s
Spiral right: mean = 46.07s, std = 25.08s, min = 22s ~ max = 133s
questionnaires: mean = 326.35s, std = 202.45s, min = 147s ~ max = 1532s
